In [26]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\tuank\INF6018\Gender_Statistics\06bb6eec-395d-4d26-972c-e26a539c43e7_Data.csv')

# Define categories for grouping related indicators
gender_equality_indicators = {
    'Labor': [
        'Ratio of female to male labor force participation rate (%) (modeled ILO estimate)',
        'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
        'Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)'
    ],
    'Employment': [
        'Employment to population ratio, 15+, female (%) (modeled ILO estimate)',
        'Employment to population ratio, 15+, male (%) (modeled ILO estimate)',
        'Unemployment, female (% of female labor force) (modeled ILO estimate)',
        'Unemployment, male (% of male labor force) (modeled ILO estimate)' 
    ],
    'Law': [
        'Women Business and the Law Index Score (scale 1-100)',
        'Women, Business and the Law: Pay Indicator Score (scale 1-100)',
        'Women, Business and the Law: Workplace Indicator Score (scale 1-100)',
        'Women, Business and the Law: Mobility Indicator Score (scale 1-100)'
    ],
    'Education': [
        'Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)',
        'Educational attainment, at least completed upper secondary, population 25+, male (%) (cumulative)',
        'Educational attainment, at least completed post-secondary, population 25+, female (%) (cumulative)',
        'Educational attainment, at least completed post-secondary, population 25+, male (%) (cumulative)',
        'Educational attainment, at least Bachelor\'s or equivalent, population 25+, female (%) (cumulative)',
        'Educational attainment, at least Bachelor\'s or equivalent, population 25+, male (%) (cumulative)'
    ],
    'Economic': [
        'GDP per capita (constant 2010 US$)',
        'GNI per capita, Atlas method (current US$)',
        'GNI per capita, PPP (current international $)'
    ]
}

# Flatten the list of indicators
all_indicators = [item for sublist in gender_equality_indicators.values() for item in sublist]

# Filter for the selected indicators and the year 2022
df_filtered = df[df['Series Name'].isin(all_indicators)]

# Pivot the DataFrame
df_pivoted = df_filtered.pivot(
    index='Country Name',
    columns='Series Name',
    values='2022 [YR2022]'
)

# Convert values to numeric and handle missing values
df_pivoted = df_pivoted.apply(pd.to_numeric, errors='coerce')
df_pivoted = df_pivoted.dropna(thresh=int(len(df_pivoted.columns) * 0.8)) # Keep countries with at least 80% data (20 columns * 0.8 = 16)

# Get all indicators from categories we're interested in
selected_indicators = []
for category in ['Education', 'Labor', 'Employment']:
    selected_indicators.extend(gender_equality_indicators[category])
    
# df_pivoted[df_pivoted.isna().any(axis=1)] # Check which countries contain NaN values

# Function to check if a row has less than 3 NaNs in the selected indicators
def less_than_two_nans(row):
    return row[selected_indicators].isna().sum() < 3

# Apply the function to filter the DataFrame
df_filtered = df_pivoted[df_pivoted.apply(less_than_two_nans, axis=1)]

# Define the columns to be updated 
columns_to_update = [
    'Educational attainment, at least completed post-secondary, population 25+, female (%) (cumulative)',
    'Educational attainment, at least completed post-secondary, population 25+, male (%) (cumulative)'
]

# Create dictionary with replacements for "Educational attainment, at least completed post-secondary, population 25+" columns
replacements = {
    'Afghanistan': [2.5354, 6.0449], # Values for female and male columns
    'Thailand': [27.3435, 26.4122],
    'United States': [49.8460, 45.8285],
    'Uzbekistan': [60.5720, 68.5913],
    'West Bank and Gaza': [30.7994, 27.7920]
}

df_new = df_filtered.copy()

# Replace NaN values
for country, values in replacements.items():
    df_new.loc[df_new.index == country, columns_to_update] = values
    
# Replace NaN values for "Educational attainment, at least Bachelor\'s or equivalent"
df_new.loc[
    df_new.index == 'Hong Kong SAR, China', 
    ['Educational attainment, at least Bachelor\'s or equivalent, population 25+, female (%) (cumulative)',
     'Educational attainment, at least Bachelor\'s or equivalent, population 25+, male (%) (cumulative)']
] = [19.67092676, 28.22598651]

# Calculate Gender Parity Indices for Labor Force Participation (GPLFP), Employment-to-Population Ratio (GPETP) and Unemployment Labor Force Rate (GPULF):

df_new['GPLFP'] = df_new['Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)'] / df_new['Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)'] * 100
df_new['GPETP'] = df_new['Employment to population ratio, 15+, female (%) (modeled ILO estimate)'] / df_new['Employment to population ratio, 15+, male (%) (modeled ILO estimate)'] * 100
df_new['GPULF'] = df_new['Unemployment, male (% of male labor force) (modeled ILO estimate)'] / df_new['Unemployment, female (% of female labor force) (modeled ILO estimate)'] * 100

# Calculate Economic Opportunities Gender Equality Index (EOGEI)
df_new['EOGEI'] = (df_new['GPLFP'] + df_new['GPETP'] + df_new['GPULF']) / 3

# Calculate Educational Attainment Gender Parity Indices for secondary, post-secondary and bachelor's level education
df_new['EAGPI_secondary'] = df_new['Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)'] / df_new['Educational attainment, at least completed upper secondary, population 25+, male (%) (cumulative)']
df_new['EAGPI_postsec'] = df_new['Educational attainment, at least completed post-secondary, population 25+, female (%) (cumulative)'] / df_new['Educational attainment, at least completed post-secondary, population 25+, male (%) (cumulative)']
df_new['EAGPI_bachelor'] = df_new['Educational attainment, at least Bachelor\'s or equivalent, population 25+, female (%) (cumulative)'] / df_new['Educational attainment, at least Bachelor\'s or equivalent, population 25+, male (%) (cumulative)']  

# Calculate overall Educational Attainment Gender Parity Indices
df_new['EAGPI_overall'] = (df_new['EAGPI_secondary'] + df_new['EAGPI_postsec'] + df_new['EAGPI_bachelor']) / 3

# Calculate pairwise differences
countries = df_new.index.tolist()
n_countries = len(countries)

# Create a dictionary to map countries to node IDs
country_to_id = {country: i+1 for i, country in enumerate(countries)}

# Create nodes dictionary
nodes = list(df_new.index)
nodes_dict = {country: index + 1 for index, country in enumerate(nodes)}
n_nodes = len(nodes)

df_columns=df_new['EOGEI'].copy()

# Create output
with open("gender_equality.net", "w") as fout:
    # Write vertices
    fout.write(f"*Vertices {n_nodes}\n")
    for node_label, node_id in nodes_dict.items():
        fout.write(f"{node_id} \"{node_label}\"\n")

    # Write edges
    fout.write("*Edges\n")
    for i in range(len(df_columns)):
        for j in range(i + 1, len(df_columns)):
            country1 = df_columns.index[i]
            country2 = df_columns.index[j]
            eogei1 = df_columns.iloc[i]
            eogei2 = df_columns.iloc[j]
            
            # Calculate the absolute difference
            diff = abs(eogei1 - eogei2)
            
            # Define the maximum allowed difference for connection
            max_diff = 7 # This can be adjusted based on your needs
            
            if diff <= max_diff:
                # Calculate weight: inverse of the difference, normalized
                weight = 1 - (diff / max_diff) 
                fout.write(f"{nodes_dict[country1]} {nodes_dict[country2]} {weight:.4f}\n")

# Write vertices to CSV
df_vertices = pd.DataFrame({'node_id': nodes_dict.values(), 'node_label': nodes_dict.keys()})
df_vertices.to_csv("gender_equality_vertices.csv", sep=",", index=False)

df_final = df_new.copy()

# Abbreviations for each indicator
abbreviations = {'Labor': {'Ratio of female to male labor force participation rate (%) (modeled ILO estimate)': 'LFP_Ratio',
                           'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)': 'LFP_Female',
                           'Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate)': 'LFP_Male'},
                 'Employment': {'Employment to population ratio, 15+, female (%) (modeled ILO estimate)': 'EMPLOYMENT_Female',
                                'Employment to population ratio, 15+, male (%) (modeled ILO estimate)': 'EMPLOYMENT_Male',
                                'Unemployment, female (% of female labor force) (modeled ILO estimate)': 'UNEMPLOYMENT_Female',
                                'Unemployment, male (% of male labor force) (modeled ILO estimate)': 'UNEMPLOYMENT_Male'},
                 'Law': {'Women Business and the Law Index Score (scale 1-100)': 'WBL_Index',
                         'Women, Business and the Law: Pay Indicator Score (scale 1-100)': 'WBL_PIS',
                         'Women, Business and the Law: Workplace Indicator Score (scale 1-100)': 'WBL_WIS',
                         'Women, Business and the Law: Mobility Indicator Score (scale 1-100)': 'WBL_MIS'},
                 'Education': {'Educational attainment, at least completed upper secondary, population 25+, female (%) (cumulative)': 'EA_SECONDARY_Female',
                               'Educational attainment, at least completed upper secondary, population 25+, male (%) (cumulative)': 'EA_SECONDARY_Male',
                               'Educational attainment, at least completed post-secondary, population 25+, female (%) (cumulative)': 'EA_POSTSEC_Female',
                               'Educational attainment, at least completed post-secondary, population 25+, male (%) (cumulative)': 'EA_POSTSEC_Male',
                               "Educational attainment, at least Bachelor's or equivalent, population 25+, female (%) (cumulative)": 'EA_BACHELOR_Female',
                               "Educational attainment, at least Bachelor's or equivalent, population 25+, male (%) (cumulative)": 'EA_BACHELOR_Male'},
                 'Economic': {'GDP per capita (constant 2010 US$)': 'GDP',
                              'GNI per capita, Atlas method (current US$)': 'GNI_Atlas',
                              'GNI per capita, PPP (current international $)': 'GNI_PPP'}}

# Flatten the abbreviations dictionary
flat_abbreviations = {indicator: abbr for category in abbreviations.values() for indicator, abbr in category.items()}

# Rename the columns in the DataFrame
df_final = df_final.rename(columns=flat_abbreviations)
#df_final = df_final.drop(columns=['LFP_Female', 'LFP_Male', 'EMPLOYMENT_Female', 'EMPLOYMENT_Male', 'UNEMPLOYMENT_Female', 'UNEMPLOYMENT_Male',
#                                  'EA_SECONDARY_Female', 'EA_SECONDARY_Male', 'EA_POSTSEC_Female', 'EA_POSTSEC_Male', 'EA_BACHELOR_Female',
#                                  'EA_BACHELOR_Male', 'EAGPI_secondary', 'EAGPI_postsec', 'EAGPI_bachelor', 'GPLFP', 'GPETP', 'GPULF', 'EOGEI'])

df_economic = df_final[['GDP','GNI_Atlas','GNI_PPP']]
df_education = df_final[['EAGPI_overall']]

# Write data frame to csv to be able to analyze further
df_final.to_csv("gender_statistics_output.csv", decimal = ",", sep=",", float_format='%.2f')
df_economic.to_csv("gender_statistics_economic_output.csv", decimal = ".", sep=",", float_format='%.2f')
df_education.to_csv("gender_statistics_education_output.csv", decimal = ".", sep=",", float_format='%.2f')